# Download last.fm listening history

  - To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)
  - My last.fm page: http://www.last.fm/user/gboeing
  - API documentation: http://www.last.fm/api
  - For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast
  
This tool separately downloads your all-time most played tracks, artists, and albums. Then it downloads all of your scrobbles in order of recency. Each of these 4 data sets are saved to separate CSV files. It downloads the "all-time most" data separately because (at least for my data) my massive iTunes history scrobble-upload in 2007 is included in the all-time most played tracks/artists/albums but is excluded from the recent tracks API endpoint. For accurate analysis of my all-time scrobbles, I need to look at those separate all-time lists, or else 4 years of listening history (from iTunes) are ignored in the calculations.

In the first cell, replace the "from keys import..." line of code with two new lines of code (replace placeholder values with your actual values):

```python
key=YOUR-LASTFM-API-KEY
username=YOUR-LASTFM-USERNAME
```

In [1]:
import requests, json, time, pandas as pd
from keys import lastfm_api_key as key, lastfm_user_name as username

In [2]:
# how long to pause between consecutive API requests
pause_duration = 0.2

## First get your all-time most played tracks, artists, and albums

In [3]:
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 0 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at

In [4]:
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('data/lastfm_top_tracks.csv', index=None, encoding='utf-8')
top_tracks.head()

,artist,track,play_count
0,Zero 7,In the Waiting Line,633
1,Bill Withers,Ain't No Sunshine,303
2,Frou Frou,Let Go,280
3,Andrew Belle,All Those Pretty Lights,237
4,Clint Mansell,Death Is the Road to Awe,195


In [5]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('data/lastfm_top_artists.csv', index=None, encoding='utf-8')
top_artists.head()

,artist,play_count
0,A.R. Rahman,1244
1,Coldplay,872
2,Zero 7,695
3,Two Door Cinema Club,618
4,Clint Mansell,438


In [6]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('data/lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums.head()

,artist,album,play_count
0,The Doors,The Very Best of The Doors,303
1,Bill Withers,Notting Hill OST,300
2,Red Hot Chili Peppers,Stadium Arcadium,299
3,Clint Mansell,The Fountain,288
4,Phoenix,Wolfgang Amadeus Phoenix,246


## Now get all your scrobbles

Last.fm provides this 'recenttracks' API method to get 'all' scrobbles. However, it seems to be pretty spotty for data from circa 2007. The best way to determine top tracks, artists, albums is with the cells above. However, the code below retrieves time series data of all scrobbles (but with the caveat of spotty data from 2007 and earlier).

Sample URL: https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=gboeing&api_key={}&limit=1&extended=0&page=1&format=json

In [10]:
def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = int(response[method]['@attr']['totalPages'])
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print('{} total pages to retrieve'.format(total_pages))
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print(page, end=' ')
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [ ]:
# get all scrobbled tracks ever, in order of recency (pages=0 to get all)
scrobbles = get_scrobbles(pages=0)

In [9]:
# save the dataset
scrobbles.to_csv('data/lastfm_scrobbles.csv', index=None, encoding='utf-8')
print('{:,} total rows'.format(len(scrobbles)))
scrobbles.head()

17,503 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime
0,Blackbird Blackbird,46e32991-1e7b-4575-9206-8e6519c97c14,Summer Heart,97c3ab39-fc2d-4596-a1d0-5690cc80c253,Hawaii,50e6dad8-1efd-4c96-b370-6a2a50b69df3,1536407564,2018-09-08 11:52:44
1,Bungalow,a3a1c085-918b-4243-85f4-fd23f53e41e5,Loving It - Single,,Loving It,,1536406547,2018-09-08 11:35:47
2,Bungalow,a3a1c085-918b-4243-85f4-fd23f53e41e5,Loving It - Single,,Loving It,,1536406303,2018-09-08 11:31:43
3,Bungalow,a3a1c085-918b-4243-85f4-fd23f53e41e5,Jay-Z - Single,,Jay-Z,,1536406071,2018-09-08 11:27:51
4,Bungalow,a3a1c085-918b-4243-85f4-fd23f53e41e5,Jay-Z - Single,,Jay-Z,,1536405789,2018-09-08 11:23:09
